# Louis Peyre Explorer - Colab
Notebook prêt à l'emploi pour lancer le pipeline sur des images stockées dans Google Drive.

## 1) Activer le GPU
Dans Colab: `Runtime > Change runtime type > T4 GPU` (ou équivalent).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content
!rm -rf /content/louis-peyre-explorer
!git clone https://github.com/gpeyre/louis-peyre-explorer.git
%cd /content/louis-peyre-explorer
!pip install -q --upgrade pip
!pip install -q -r requirements.txt

## 2) Vérifier le dossier d'images sur Drive
Les images doivent être dans: `/content/drive/MyDrive/louis-peyre-images`

In [ ]:
!ls -la /content/drive/MyDrive/louis-peyre-images | head -n 30

## 3) Lancer le pipeline
Astuce: commencez avec `max_images=5` pour valider rapidement le setup.

In [ ]:
max_images = 20
sam_model_id = "facebook/sam-vit-huge"  # remplacer par facebook/sam-vit-base si OOM

In [ ]:
!python pipeline.py \
  --drive_folder louis-peyre-images \
  --max_images {max_images} \
  --sam_model_id {sam_model_id}

## 4) Résultats
Fichiers générés sur Drive:
- `/content/drive/MyDrive/louis-peyre-images/crops/`
- `/content/drive/MyDrive/louis-peyre-images/provenance.csv`

In [ ]:
!ls -la /content/drive/MyDrive/louis-peyre-images/crops | head -n 30
!python - <<'PY'
import pandas as pd
p='/content/drive/MyDrive/louis-peyre-images/provenance.csv'
df=pd.read_csv(p)
print(df.head())
print('Nombre de lignes:', len(df))
PY